In [1]:
import sqlite3
import xlrd, xlwt
from tqdm import tqdm_notebook
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import re
import csv


import numpy as np
import pandas as pd

# ML moduls
import keras
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec
from tqdm import tqdm
from nltk.corpus import stopwords
import nltk.data
import logging
import _pickle as cPickle
from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Начальная обработка текста
def review_to_wordlist(review, remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #
    # 2. Remove non-letters
    #review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)


In [3]:
#Токенизация
# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    # Function to split a review into parsed sentences. Returns a
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            list_of_words = review_to_wordlist(raw_sentence, remove_stopwords)
            if len(list_of_words) > 0:
                sentences.append(list_of_words)
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences


In [4]:
# Составляем вектора
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    #
    # Index2word is a list that contains the names of the words in
    # the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    #
    # Divide the result by the number of words to get the average
    #print(featureVec)
    if(nwords == 0):
        global count_null_nwords 
        count_null_nwords = 0
        count_null_nwords += 1
        nwords += 1
    featureVec = np.divide(featureVec,nwords)
    #print(featureVec)
    return featureVec


In [5]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate
    # the average feature vector for each one and return a 2D numpy array
    #
    # Initialize a counter
    counter = 0
    
    #
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    #
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       #if (counter%1000.) == (0.):
        #   print ("Review %d of %d" % (counter, len(reviews)))
       #
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       
       #
       # Increment the counter
       counter = counter + 1

    return reviewFeatureVecs


In [6]:
def csv_writer(data, path):
    with open(path, "w", newline='', encoding="utf8") as csv_file:
        writer = csv.writer(csv_file, delimiter='\t')
        writer.writerow("id_tweet, sentiment, us_id, followers".split(", "))
        for line in data:
            writer.writerow(line)


def csv_writer1(data, path):
    with open(path, "a", newline='', encoding="utf8") as csv_file:
        writer = csv.writer(csv_file, delimiter='\t')
        for line in data:
            writer.writerow(line)


In [13]:
def lol():
    path = "C:/Users/Users/Maxim/Nero/rfirst.csv"
    csv_writer(results1,path)

    # Чтение файлов и разбивка
    concat_data = pd.read_csv(r"C:\Users\Users\Maxim\Nero\rfirst.csv", header=0, delimiter="\t", quoting=3, encoding = "ISO-8859-1")

    train = concat_data.sample(frac=0.0)
    test = concat_data.loc[~concat_data.index.isin(train.index)]
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)

    train.to_csv( "C:/Users/Users/Maxim/Nero/TwitterAnalyse/twit2vec/trainlol.csv", index=False, sep='\t', escapechar='\\', quoting=3 )
    test.to_csv( "C:/Users/Users/Maxim/Nero/TwitterAnalyse/twit2vec/testlol.csv", index=False, sep='\t', escapechar='\\', quoting=3 )

    colmn_review ="sentiment"

    # Verify the number of reviews that were read (100,000 in total)
    #printprint ("Read %d labeled train reviews, %d labeled test reviews, " % (train[colmn_review].size, test[colmn_review].size))

    # Load the punkt tokenizer
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


    sentences = []  # Initialize an empty list of sentences

    #print ("Parsing sentences from training set")
    for review in train[colmn_review]:
        sentences += review_to_sentences(review, tokenizer)


    # Import the built-in logging module and configure it so that Word2Vec
    # creates nice output messages

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
        level=logging.INFO)

    # Set values for various parameters
    num_features = 300    # Word vector dimensionality
    min_word_count = 10   # Minimum word count
    num_workers = 4       # Number of threads to run in parallel
    context = 15          # Context window size
    downsampling = 1e-4   # Downsample setting for frequent words


    # ****************************************************************
    model=word2vec.Word2Vec.load("C:/Users/Users/Maxim/Nero/100features_10minwords_10context")
    #print(model)
    # ****************************************************************
    # Calculate average feature vectors for training and testing sets,
    # using the functions we defined above. Notice that we now use stop word
    # removal.

    clean_train_reviews = []
    for review in train[colmn_review]:
        clean_train_reviews.append( review_to_wordlist( review,remove_stopwords=True ))


    count_null_nwords=0
    trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )
    #print('count_null_nwords: ', count_null_nwords)


    #print ("Creating average feature vecs for test reviews")
    clean_test_reviews = []
    for review in test[colmn_review]:
        clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

    testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )


    forest = RandomForestClassifier( n_estimators = 100 )

    # load it again
    with open('C:/Users/Users/Maxim/Nero/my_dumped_classifier.pkl', 'rb') as fid:
        forest = cPickle.load(fid)

    # Test & extract results
    result = forest.predict( testDataVecs )

    # Write the test results
    output = pd.DataFrame( data={"id":test["id_tweet"], "review":result,"us_id":test["us_id"],"followers":test["followers"]} )
    output.to_csv("C:/Users/Users/Maxim/Nero/TwitterAnalyse/twit2vec/Word2Vec_AverageVectors_r.csv", index=False, quoting=3)
    #print("OK")

    #КОНЕЦ ПЕРВОЙ НЕЙРОСЕТИ

    # Создаем курсор - это специальный объект который делает запросы и получает их результаты
    cursor3 = conn.cursor()
    cursor4 = conn.cursor()
    cursor5 = conn.cursor()

    fmt = '%Y-%m-%d %H:%M:%S'
    fmt2 = '%d.%m.%Y %H:%M:%S'

    # Делаем SELECT запрос к базе данных, используя обычный SQL-синтаксис
    results3 = cursor3.fetchall()
    ZZZ=0

    
    #ДАННЫЕ ИЗ ПЕРВОЙ НЕЙРОСЕТИ В БД
    cursor3 = conn.cursor()
    cursor3.execute("""DELETE FROM rner1""")
    conn.commit()
    z=0
    with open('C:/Users/Users/Maxim/Nero/TwitterAnalyse/twit2vec/Word2Vec_AverageVectors_r.csv', encoding="utf8") as tsvfile:
        reader = csv.reader(tsvfile, delimiter=',')
        for row in reader:
            if(z>0):
                cursor5.execute("insert into rner1 (followers,id_tweet,review,us_id)values (?,?,?,?)",
                        (row[0],row[1],row[2],row[3]))
            z+=1
    conn.commit()

    
    #Создаю массив для ввода во вторую нейросеть
    cursor4.execute("""SELECT * FROM rner1""")
    results4 = cursor4.fetchall()
    a=0
    i=0
    o=0
    Z = np.zeros((2999,1))
    for i in range(len(results4)):
        while a<1499:
            Z[2*a]=results4[a][1]
            Z[2*a+1]=results4[a][3]
            a+=1

    Z=Z.T
    X_test = Z

    
    #ПРОВЕРЯЮ РЕЗУЛЬТАТ
    x = model_load.predict(X_test)
    for i in range(len(x)):
        print("X=%s, Predicted=%s" % (X_test[i], x[i]))
        #print(datetime.utcnow())
        y = x[i]
    if y[0]==1:
        r = 1
    elif y[1]==1:
        r=0
    else:
        r=-1
        
    return(r)


In [14]:
# Создаем соединение с нашей базой данных
# В нашем примере у нас это просто файл базы
conn = sqlite3.connect('C:/Users/Users/Maxim/Nero/Chinook_Sqlite.sqlite')

# Создаем курсор - это специальный объект который делает запросы и получает их результаты
cursor1 = conn.cursor()
cursor1.execute("""SELECT id_tweet, review, us_id, followers FROM rtwi ORDER BY id_tweet DESC LIMIT 1500""")
results1 = cursor1.fetchall()


path_save_itog = 'C:/Users/Users/Maxim/Nero/itog1.csv'
url111 = 'https://coinmarketcap.com/ru/currencies/bitcoin/'

# ML model
model_load = load_model('C:/Users/Users/Maxim/Nero/my_model.h5')


In [ ]:
while True:
    
    page111 = requests.get(url111)
    soup11 = BeautifulSoup(page111.content, 'html.parser')
    ART = soup11.find('span', class_="h2 text-semi-bold details-panel-item--price__value")
    price111 = ART.string
    dnnn = str(lol()) + ',' + str(datetime.now()) + ',' + str(price111)
    csv_writer1(dnnn, path_save_itog)
    print(dnnn)

    time.sleep(1500)
    

2018-11-01 12:27:00,843 : INFO : loading Word2Vec object from C:/Users/Users/Maxim/Nero/100features_10minwords_10context
2018-11-01 12:27:01,366 : INFO : loading wv recursively from C:/Users/Users/Maxim/Nero/100features_10minwords_10context.wv.* with mmap=None
2018-11-01 12:27:01,368 : INFO : setting ignored attribute vectors_norm to None
2018-11-01 12:27:01,369 : INFO : loading vocabulary recursively from C:/Users/Users/Maxim/Nero/100features_10minwords_10context.vocabulary.* with mmap=None
2018-11-01 12:27:01,371 : INFO : loading trainables recursively from C:/Users/Users/Maxim/Nero/100features_10minwords_10context.trainables.* with mmap=None
2018-11-01 12:27:01,373 : INFO : setting ignored attribute cum_table to None
2018-11-01 12:27:01,375 : INFO : loaded C:/Users/Users/Maxim/Nero/100features_10minwords_10context
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__g

X=[1.01933199e+18 1.05789673e+18 1.01933199e+18 ... 1.09423855e+09
 1.05789434e+18 0.00000000e+00], Predicted=[1. 0. 0.]
1,2018-11-01 12:27:06.076759,6403.97
